# Lecture des informations d'une carte d'identité
L'objectif de ce script est d'arriver à extraire les informations d'une carte d'identité à partir d'une photo.
Si l'arrière plan est uni, alors le script fonctionnera correctement. En revanche, si le fond est composé de couleurs bien distinctes et que ces couleurs collent la carte, le script ne détectera pas correctement les contours et corrigera alors très mal la perspective ou éventuellement en créera ce qui la rendra illisible pour pytesseract

In [1]:
import numpy as np
import cv2
from os import makedirs
from os import path
import pytesseract

## Transformation de l'image
Teste la validité des contours trouvés par opencv. Si le contour fait moins de 20% de la taille de l'image originale on considèrera que le contour n'est pas validé car trop petit et ce contour sera sûrement celui d'une des lettres sur la carte d'identité ou un détail inutile. En revanche si le contour fait plus de 95% de l'image on supposera qu'opencv n'a pas trouvé les contours de la carte d'identité ou alors la correction de perspective est inutile car il n'y a pas de perspective sur la photo.

In [12]:
def testContourValidity(contour, full_width, full_height):
    """On prend une valeur max du contour de 95% de la photo originale"""
    max_threshold = 0.95
    """On prend une valeur min du contour de 20% de la photo originale pour eviter qu'il ne considère 
    tous les petits contours créés par les lettres ainsi que par les sections dans la carte. Cette valeur 
    peut être ajustée en fonction de la photo mais globalement elle fonctionne bien"""
    min_threshold = 0.2
    min_area = full_width * full_height * min_threshold
    max_area = full_width * full_height * max_threshold
    max_width = full_width * max_threshold
    max_height = full_height * max_threshold  # *
    min_width = min_threshold * full_width
    min_height = min_threshold * full_height
    size = cv2.contourArea(contour)
    # Area
    if size < min_area:
        return False
    if size > max_area:
        return False

    """J'utilise la fonction min rect area car elle prend le rectangle orienté qui englobe tout le contour ce 
    qui permet d'avoir une aire plus réaliste si les contours ne sont pas exacts 
    (voir cellule du dessous pour précision)"""
    rect = cv2.minAreaRect(contour)

    box = cv2.boxPoints(rect)

    box = np.int0(box)
    (tl, tr, br, bl) = sort_points(box)
    """calcul de la taille du rectangle trouvé"""
    box_width = int(((br[0] - bl[0]) + (tr[0] - tl[0])) / 2)
    box_height = int(((br[1] - tr[1]) + (bl[1] - tl[1])) / 2)
    """le and peut etre laissé mais dans certain cas il sera moins performant que cette organisation pour les if"""
    if box_width < min_width: #and box_height < min_height:
        return False
    if box_height < min_height:
        return False
    if box_width > max_width: #and box_height > max_height:
        return False
    if box_height > max_height:
        return False
    return True


<img src="files/explication.jpg">

In [24]:
def find_square(im, f):
    # Width and height for validity check
    h = np.size(im, 0)
    w = np.size(im, 1)

    """shadow removal"""
    img = cv2.imread('source_images/' + f, -1)

    rgb_planes = cv2.split(img)

    result_norm_planes = []

    for plane in rgb_planes:
        dilated_img = cv2.dilate(plane, np.ones((7, 7), np.uint8))

        bg_img = cv2.medianBlur(dilated_img, 21)

        diff_img = 255 - cv2.absdiff(plane, bg_img)

        norm_img = cv2.normalize(diff_img, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8UC1)

        result_norm_planes.append(norm_img)

    result_norm = cv2.merge(result_norm_planes)
    """save the shadow removed picture"""
    debug_image(result_norm, 'preprocess_shadow', f)
    #result_norm=cv2.imread('source_images/' + f, 0)
    result_norm = cv2.Canny(result_norm, 100, 255)
    """save the canny picture"""
    debug_image(result_norm, 'preprocess_canny', f)
    blur = cv2.GaussianBlur(result_norm, (5, 5), 0)
    debug_image(blur, 'preprocess_blur', f)
    ret, thresh = cv2.threshold(blur, 100, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    """Tentative d'utilisation des Hough lines mais pas très concluant car la carte d'identité 
    a trop de motifs à l'intérieur cependant peut sûrement être utilisé pour remplir la carte 
    d'une couleur et détecter les contours"""
    linesP = cv2.HoughLinesP(thresh, 1500, np.pi /360, 50, None, 100, 50)

    if linesP is not None:
        for i in range(0, len(linesP)):
            l = linesP[i][0]
            cv2.line(result_norm, (l[0], l[1]), (l[2], l[3]), (255, 255, 255), 3, cv2.LINE_AA)
    cv2.imwrite('lines.jpg',result_norm)
    debug_image(thresh, 'preprocess_thresh', f)
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    im_debug = im.copy()
    """Recherche du contour le plus grand"""
    max = None
    i = 0
    for x in contours:
        if testContourValidity(x, w, h ):
            i += 1
            im_debug = cv2.drawContours(im_debug, [x], -1, (0, 255, 0), 3)
            if max is None or cv2.contourArea(max) > cv2.contourArea(x):
                max = x

    """Premiere tentative d'approximation des contours en une forme géométrique mais cette fonction ne fait que 
    des rectangles ce qui ne permet pas de corriger la perspective elle permet seulement de corriger une rotation"""
    #rect = cv2.minAreaRect(max)
    """Je ne cherche pas un polygone fermé je cherche uniquement à trouver les coins. S'il détecte les 
    contours en entier tant mieux mais ce n'est pas obligatoire"""
    rect=cv2.approxPolyDP(max, 0.01*cv2.arcLength(max, False), False)
    im_debug = cv2.drawContours(im_debug, [max], -1, (0, 0, 255), 3)
    """contours trouvés par opencv"""
    debug_image(im_debug, 'contour', f)
    """approximation des contours par le polygone"""
    debug_image(cv2.drawContours(im.copy(),rect, -1,(0,255,0),10),'polygone',f)
    #box = cv2.boxPoints(rect)
    #box = np.int0(box)
    """transformation du tableau rect en un tableau du format adéquat pour etre traité plus tard"""
    box=[]
    for i in range(len(rect)):
        box.append(rect[i][0])
    box=np.array(box)
    return box

In [25]:
def diff(a, b):
    return (a - b) ** 2

In [26]:
def adist(a, b):
    return np.sqrt(diff(a[0], b[0]) + diff(a[1], b[1]))


In [27]:
def max_distance(a1, a2, b1, b2):
    dist1 = adist(a1, a2)
    dist2 = adist(b1, b2)
    if int(dist2) < int(dist1):
        return int(dist1)
    else:
        return int(dist2)

In [28]:
def sort_points(pts):
    ret = np.zeros((4, 2), dtype="float32")
    sumF = pts.sum(axis=1)
    diffF = np.diff(pts, axis=1)
    ret[0] = pts[np.argmin(sumF)]
    ret[1] = pts[np.argmin(diffF)]
    ret[2] = pts[np.argmax(sumF)]
    ret[3] = pts[np.argmax(diffF)]
    return ret


In [29]:
def fix_perspective(image, pts):
    """trouve les 4 coins du polygone"""
    (tl, tr, br, bl) = sort_points(pts)
    """cherche la hauteur max entre les coins qui deviendra la hauteur de la nouvelle image"""
    maxW = max_distance(br, bl, tr, tl)
    """cherche la longeur max entre les coins qui deviendra la largeur de la nouvelle image"""
    maxH = max_distance(tr, br, tl, bl)
    """tableau contenant les coordonnées des 4 coins dans l'image de destination"""
    dst = np.array([[0, 0], [maxW - 1, 0], [maxW - 1, maxH - 1], [0, maxH - 1]], dtype="float32")
    """corrige la perspective"""
    transform = cv2.getPerspectiveTransform(np.array([tl, tr, br, bl], dtype="float32"), dst)
    fixed = cv2.warpPerspective(image, transform, (maxW, maxH))
    return fixed


In [30]:
def debug_image(img, extra_path, filename):
    fpath = "debug/" + extra_path + "/"
    if not path.isdir(fpath):
        makedirs(fpath)
    cv2.imwrite(fpath + filename, img)


In [32]:
f = 'Carte-didentite.jpg'
filename = "source_images/" + f
img = cv2.imread(filename)

square = find_square(img, f)

im_debug = cv2.drawContours(img.copy(), [square], -1, (0, 255, 0), 3)
debug_image(im_debug, "selected_contour", f)

img = fix_perspective(img, square)
filename2 = "fixed " + f
cv2.imwrite(filename2, img)

True

## Lecture extraction des informations
on lit les informations de 2 manières "différentes", la 2e semble plus efficace généralement mais dans certains cas la première est plus efficace

In [21]:
import pytesseract
text1 = pytesseract.image_to_string(img, lang="fra", config="--psm 6 --oem 2")
text2 = pytesseract.image_to_data(img, lang="fra", output_type='data.frame')
print(text1)
text = text2[text2.conf != -1]
lines = text.groupby('block_num')['text'].apply(list)
print()
for i in range(len(lines)):
    print("niveau", i, ": ", lines.iloc[i])

TesseractNotFoundError: tesseract is not installed or it's not in your PATH. See README file for more information.